In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Trainer, TrainingArguments
import torch
import os
import pandas as pd
from tqdm import tqdm
from datasets import Dataset
from fuzzywuzzy import fuzz
import evaluate
import csv
import numpy as np
import torch.nn.functional as F
from sklearn.metrics import top_k_accuracy_score, ndcg_score
import random

/mounts/u-spa-d4/grad/mfe261/Projects/MobileConvRec/envs/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/mounts/u-spa-d4/grad/mfe261/Projects/MobileConvRec/envs/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
def get_first_five_words(sentence):
    words = sentence.split()  # Split the sentence into a list of words
    return " ".join(words[:10])  # Join the first 5 words back into a string

In [4]:
input_file = "/u/spa-d4/grad/mfe261/Projects/MobileConvRec/dataset/amazon_garden/splits/train.jsonl"
df_recommender_train = pd.read_json(input_file, lines=True)
for _, row in df_recommender_train.iterrows():
    row["recommended_product"]["product_name"] = row["recommended_product"]["product_name"].lower()

In [5]:
df_recommender_train

,user_id,user_previous_interactions,recommended_product,negative_recommended_product,turns
0,AFVJZUNAWT6ZLSES4FT3AYBXCJKA,[],{'product_name': 'lodge pre-seasoned cast iron...,[],"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
1,AEDYOQKGVEHNISQYBMND2ZZHYPDA,[],{'product_name': 'weber 825020 22 performer ch...,[{'product_name': 'Char-Broil 463377319 Perfor...,"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
2,AFWA2KTWWVNOP7T2EYX6VAOIGKEQ,[],{'product_name': 'char- broil standard portabl...,[{'product_name': 'Char-Broil Classic 280 2-Bu...,"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
3,AEJ6HUUENHAFSSJTEOI4MVNQJZ4A,[],{'product_name': 'great states 304-14 14-inch ...,[{'product_name': 'Sun Joe MJ500M 16-Inch Manu...,"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
4,AGYZ3XAFPLGM3Z5KITKKKU4HJQKQ,[],"{'product_name': 'weber chimney starter', 'par...",[{'product_name': 'homenote Rapid Charcoal Chi...,"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
...,...,...,...,...,...
8528,AEQFUXVWNBKN3OWA6GBT54FPS4RQ,[{'product_name': 'Weber 6678 Large Original G...,{'product_name': 'nocry home & gardening knee ...,[{'product_name': 'InSassy Garden Kneeler Pad ...,"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
8529,AGJDABOTXPJY6M42ECWU2UHMILPQ,[{'product_name': 'VIVOSUN 6.5 Inch Gardening ...,{'product_name': 'super soil autoflower concen...,[],"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
8530,AFNKT54HHGQTCFS3GHYMTMTGOTJQ,[{'product_name': 'LogOX Hauler Ergonomic Log ...,{'product_name': 'foxfarm fx14106 752289794056...,[{'product_name': 'Compare-N-Save Concentrate ...,"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
8531,AHA3CPCZ66UFSEY5AWC4OHWMHTYA,[{'product_name': 'HOLDPEAK Digital Anemometer...,{'product_name': 'scotts turf builder weed & f...,[{'product_name': 'Scotts Turf Builder Halts C...,"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."


In [6]:
input_file = "/u/spa-d4/grad/mfe261/Projects/MobileConvRec/dataset/amazon_garden/splits/val.jsonl"
df_recommender_validation = pd.read_json(input_file, lines=True)
for _, row in df_recommender_validation.iterrows():
    row["recommended_product"]["product_name"] = row["recommended_product"]["product_name"].lower()

In [7]:
df_recommender_validation

,user_id,user_previous_interactions,recommended_product,negative_recommended_product,turns
0,AFK66DCACOXFTGEPCH6VOZFPRSCQ,[{'product_name': 'HH-Eason 6 pcs Micro Landsc...,{'product_name': 'suncast 22 gallon indoor or ...,[{'product_name': 'Keter City 30 Gallon Resin ...,"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
1,AGYUZAAY3Z7HYBSQRUPCOUQUZFZQ,[{'product_name': 'ANLEY Fly Breeze 3x5 Foot U...,{'product_name': 'vivosun 6 mil mylar film rol...,[],"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
2,AGL4MJX4J7RXZDV7LVEZKHLUW7AQ,[{'product_name': 'Gardzen 72 Pcs 5 Inch Plast...,{'product_name': 'torro products 4 pack mole r...,[{'product_name': 'YARDEC Gopher Repellent Ult...,"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
3,AG2VO5ZRX6FC42TZMI37IFUAUYCQ,[{'product_name': 'Juegoal 2 Pack Hummingbird ...,{'product_name': 'champion 8000-watt dual fuel...,[{'product_name': 'Westinghouse Outdoor Power ...,"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
4,AHQ2AHLHIS6UWQJXYNP4SQ6HS2EA,[{'product_name': 'BioAdvanced Complete Insect...,"{'product_name': 'foxfarm fx14019 fertilizer, ...",[{'product_name': 'Premium Water Soluble Ferti...,"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
...,...,...,...,...,...
1823,AGVYLUBF4ETB5QKKEKIZLSNVD6SQ,[{'product_name': 'Terro T1700SR 2 Pack 19 oz ...,{'product_name': 'honda eu2200itag 2200-watt 1...,[{'product_name': 'Honda 664240 EU2200i 2200 W...,"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
1824,AGLTUGLIGJQL4RUKOEBDOVN5MLEQ,[{'product_name': 'Fiskars Micro-Tip Pruning S...,{'product_name': 'wanapure 25pcs copper metal ...,[{'product_name': 'Vktech 5000pcs 6 x10cm Plas...,"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
1825,AH43BYN2DG2EJXREWJ6NZBKORJNQ,[],{'product_name': 'j r peters 52024 jacks class...,[{'product_name': 'J R Peters Jacks Classic No...,"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
1826,AH43BYN2DG2EJXREWJ6NZBKORJNQ,[{'product_name': 'J R Peters 52024 Jacks Clas...,{'product_name': 'jack's classic 10-30-20 blos...,[{'product_name': 'Neptune's Harvest Tomato & ...,"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."


In [8]:
model = AutoModelForSeq2SeqLM.from_pretrained(pretrained_model_name_or_path = "google/flan-t5-base")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base", additional_special_tokens=["computer:", "human:"])
model.resize_token_embeddings(len(tokenizer))
IGNORE_INDEX = -100

/mounts/u-spa-d4/grad/mfe261/Projects/MobileConvRec/envs/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [9]:
prompt_train = []
recommend_train = []
not_founds = 0

for _, row in tqdm(df_recommender_train.iterrows(), total=len(df_recommender_train)):
    prompt = ""
    found = False
    recommended = get_first_five_words(row["recommended_product"]["product_name"])
    
    for index, turn in enumerate(row["turns"]):
        if "COMPUTER" in turn:
            computer = turn["COMPUTER"]
        
            if fuzz.partial_ratio(recommended, computer.lower()) >= 95:
                prompt += "computer: I would recommend the "
                prompt_train.append(prompt)
                recommend_train.append(recommended)
                found = True
                break
            else:
                prompt += "computer: "+ computer + "\n"
        
        if "HUMAN" in turn:
            human = turn["HUMAN"]
            prompt += "human: " + human + "\n"
    
    if not found:
        not_founds += 1

print(f"Could not find {not_founds}")
print(f"len(prompt_train): {len(prompt_train)}")
print(f"len(recommend_train): {len(recommend_train)}")

            
            
prompt_encodings = tokenizer(prompt_train, padding='max_length', max_length=1024, truncation=True, return_tensors='pt')
recommend_encodings = tokenizer(recommend_train, padding='max_length', max_length=32, truncation=True, return_tensors='pt')

labels = recommend_encodings['input_ids']
labels[labels == tokenizer.pad_token_id] = IGNORE_INDEX

dataset = {
    'input_ids': prompt_encodings['input_ids'],
    'attention_mask': prompt_encodings['attention_mask'],
    'labels': labels,
}
dataset_train = Dataset.from_dict(dataset)

  0%|          | 0/8533 [00:00<?, ?it/s]

100%|██████████| 8533/8533 [01:44<00:00, 81.79it/s] 


Could not find 146
len(prompt_train): 8387
len(recommend_train): 8387


In [10]:
prompt_validation = []
recommend_validation = []
not_founds = 0

for _, row in tqdm(df_recommender_validation.iterrows(), total=len(df_recommender_validation)):
    prompt = ""
    found = False
    recommended = get_first_five_words(row["recommended_product"]["product_name"])
    
    for index, turn in enumerate(row["turns"]):
        if "COMPUTER" in turn:
            computer = turn["COMPUTER"]
            
            if fuzz.partial_ratio(recommended, computer.lower()) >= 95:
                prompt += "computer: I would recommend the "
                prompt_validation.append(prompt)
                recommend_validation.append(recommended)
                found = True
                break
            else:
                prompt += "computer: "+ computer + "\n"
        
        if "HUMAN" in turn:
            human = turn["HUMAN"]
            prompt += "human: " + human + "\n"
    
    if not found:
        not_founds += 1
        
print(f"Could not find {not_founds}")
print(f"len(prompt_validation): {len(prompt_validation)}")
print(f"len(recommend_validation): {len(recommend_validation)}")
            
            
prompt_encodings = tokenizer(prompt_validation, padding='max_length', max_length=1024, truncation=True, return_tensors='pt')
recommend_encodings = tokenizer(recommend_validation, padding='max_length', max_length=32, truncation=True, return_tensors='pt')

labels = recommend_encodings['input_ids']
labels[labels == tokenizer.pad_token_id] = IGNORE_INDEX

dataset = {
    'input_ids': prompt_encodings['input_ids'],
    'attention_mask': prompt_encodings['attention_mask'],
    'labels': labels,
}
dataset_validation = Dataset.from_dict(dataset)

  0%|          | 0/1828 [00:00<?, ?it/s]

100%|██████████| 1828/1828 [00:23<00:00, 77.05it/s]


Could not find 57
len(prompt_validation): 1771
len(recommend_validation): 1771


In [11]:
def data_collator(batch):
    input_ids, attention_mask, labels,  = [], [], []
    for sample in batch:
        input_ids.append(sample['input_ids'])
        attention_mask.append(sample['attention_mask'])
        labels.append(sample['labels'])
    max_encoder_len = max(sum(x) for x in attention_mask)
    max_decoder_len = max(sum([0 if item == IGNORE_INDEX else 1 for item in x]) for x in labels)
    return {
        'input_ids': torch.tensor(input_ids)[:, :max_encoder_len],
        'attention_mask': torch.tensor(attention_mask)[:, :max_encoder_len],
        'labels': torch.tensor(labels)[:, :max_decoder_len]
    }

In [12]:
training_args = TrainingArguments(
    output_dir="/u/spa-d4/grad/mfe261/Projects/MobileConvRec/models/new_models/amazon_garden/T5_recommender",
    num_train_epochs=5,
    # logging_steps=500,
    # logging_dir=self.cfg.logging_dir,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    save_strategy="steps",
    evaluation_strategy="steps",
    save_steps=0.3,#self.cfg.save_steps,
    eval_steps=0.3, #self.cfg.eval_steps,
    save_total_limit=3,
    gradient_accumulation_steps=3, #gradient_accumulation_steps,
    per_device_train_batch_size=4, #train_batch_size,
    per_device_eval_batch_size=4, #self.cfg.eval_batch_size,
    warmup_steps=100,
    weight_decay=0.01,
    # dataloader_drop_last=True,
    disable_tqdm=False,
    push_to_hub=False
)

trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset_train,
        eval_dataset=dataset_validation,
        data_collator=data_collator,
    )

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [13]:
trainer.train() # resume_from_checkpoint=True
trainer.save_model()

Step,Training Loss,Validation Loss
1049,3.245900,2.983224
2098,2.948500,2.895374
3147,2.812100,2.845237


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


# Load the model and test it on the test dataset

In [14]:
torch.cuda.empty_cache()

In [15]:
input_file = "/u/spa-d4/grad/mfe261/Projects/MobileConvRec/dataset/amazon_garden/splits/test.jsonl"
df_recommender_test = pd.read_json(input_file, lines=True)
for _, row in df_recommender_test.iterrows():
    row["recommended_product"]["product_name"] = row["recommended_product"]["product_name"].lower()

In [16]:
apps_training_path = "/u/spa-d4/grad/mfe261/Projects/MobileConvRec/dataset/amazon_garden/garden_df.csv"

all_apps = []
with open(apps_training_path, 'r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader:
        all_apps.append(get_first_five_words(row["title"].lower()))
        
all_apps = list(set(all_apps))

In [17]:
def candidate_creator(row):
    np.random.seed(row.name)
    selected_values = np.random.choice(np.setdiff1d( all_apps, [get_first_five_words(row["recommended_product"]["product_name"])]), 24, replace=False) # filter_candidate_apps(row["recommended_book"]["book_name"])
    random_position = np.random.randint(0, len(selected_values) + 1)
    
    return np.insert(selected_values, random_position, get_first_five_words(row["recommended_product"]["product_name"])) 

df_recommender_test['candidate'] = df_recommender_test.apply(lambda row: candidate_creator(row), axis=1)

In [18]:
prompt_test = []
recommend_test = []
candidate_books = []
true_candidate_indexes = []
not_founds = 0
for _, row in df_recommender_test.iterrows():
    candidates = []
    for index, candidate_book in enumerate(row["candidate"].tolist()):
        candidates.append(candidate_book)
        if candidate_book == get_first_five_words(row["recommended_product"]["product_name"]):
            true_candidate_index = index
    prompt = ""
    
    found = False
    recommended = get_first_five_words(row["recommended_product"]["product_name"])
    
    for index, turn in enumerate(row["turns"]):
        computer = turn["COMPUTER"]
        
        if fuzz.partial_ratio(recommended, computer.lower()) >= 95:
            prompt += "computer: I would recommend the "
            prompt_test.append(prompt)
            recommend_test.append(recommended)
            candidate_books.append(candidates)
            true_candidate_indexes.append(true_candidate_index)
            found = True
            break
        else:
            prompt += "computer: "+ computer + "\n"
        
        if "HUMAN" in turn:
            human = turn["HUMAN"]
            prompt += "human: " + human + "\n"
    
    if not found:
        not_founds += 1

print(f"Could not find {not_founds}")
print(f"Number of prompt: {len(prompt_test)}")
print(f"Number of generations: {len(recommend_test)}")
print(f"Number of candidate apps: {len(candidate_books)}")
print(f"Number of true candidate indexes: {len(true_candidate_indexes)}")

Could not find 70
Number of prompt: 1759
Number of generations: 1759
Number of candidate apps: 1759
Number of true candidate indexes: 1759


In [19]:
model = AutoModelForSeq2SeqLM.from_pretrained(pretrained_model_name_or_path = "/u/spa-d4/grad/mfe261/Projects/MobileConvRec/models/new_models/amazon_garden/T5_recommender")
model.eval()
model = model.to('cuda')
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base", additional_special_tokens=["computer:", "human:"])
model.resize_token_embeddings(len(tokenizer))
IGNORE_INDEX = -100

/mounts/u-spa-d4/grad/mfe261/Projects/MobileConvRec/envs/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [20]:
def chunk(list_of_elements, batch_size): # using this chunk function, we can split our data to multiple batches
  for i in range(0, len(list_of_elements), batch_size):
    yield list_of_elements[i:i+batch_size]

def evaluate_recommender(prompt_test, recommend_test, model, tokenizer, batch_size=8, threshold=70):
  prompt_batches = list(chunk(prompt_test, batch_size))
  generation_batches = list(chunk(recommend_test, batch_size))

  correctly_predicted = []
  for prompt_batch, generation_batch in tqdm(zip(prompt_batches, generation_batches), total = len(generation_batches)):

    inputs = tokenizer(prompt_batch, max_length=1024, truncation=True, padding="max_length", return_tensors="pt") 

    generations_predicted = model.generate(input_ids=inputs["input_ids"].to('cuda'), attention_mask=inputs["attention_mask"].to('cuda'),
                            max_new_tokens=32,
                            num_beams=8,
                            eos_token_id=tokenizer.eos_token_id,
                            pad_token_id=tokenizer.pad_token_id,
                            bos_token_id=tokenizer.bos_token_id) # length_penalty=0.8, Set length_penalty to values < 1.0 in order to encourage the model to generate shorter sequences, to a value > 1.0 in order to encourage the model to produce longer sequences.

    decoded_generations = [tokenizer.decode(generation, skip_special_tokens=True, clean_up_tokenization_spaces=True) for generation in generations_predicted]
    generation_batch = [generation for generation in generation_batch]
    
    correctly_predicted.extend([1 if fuzz.ratio(predicted, ground_truth) > threshold else 0 for predicted, ground_truth in zip(decoded_generations, generation_batch)])

  return correctly_predicted

In [21]:
correctly_predicted = evaluate_recommender(prompt_test, recommend_test, model, tokenizer, batch_size=8, threshold=70)
success_rate = sum(correctly_predicted) / len(correctly_predicted)
print("success_rate: ", success_rate)

  0%|          | 0/220 [00:00<?, ?it/s]

100%|██████████| 220/220 [04:30<00:00,  1.23s/it]

success_rate:  0.07276861853325753


In [22]:
def chunk(list_of_elements, batch_size): # using this chunk function, we can split our data to multiple batches
  for i in range(0, len(list_of_elements), batch_size):
    yield list_of_elements[i:i+batch_size]
    
def convert_to_sublists(numbers, sublist_size):
    return [numbers[i:i+sublist_size] for i in range(0, len(numbers), sublist_size)]

def recommender_rank(prompts, candidate_apps, model, tokenizer, batch_size=8):
  model.eval()
  encoder_max_length = 1024
  decoder_max_length = 32
  prompts_tokenized = tokenizer(prompts, max_length=encoder_max_length, truncation=True, padding="max_length", return_tensors="pt")
  
  input_ids_decoder = []
  attention_mask_decoder = []
  input_ids_encoder = []
  attention_mask_encoder  = []
  for index, candidate_app_elements in enumerate(candidate_apps):
    candidate_app_elements = [tokenizer.pad_token+element for element in candidate_app_elements] # adding pad token to the beginning of each candidate app
    candidate_apps_tokenized = tokenizer(candidate_app_elements, max_length=decoder_max_length, truncation=True, padding="max_length", return_tensors="pt")
    for candidate_app_index in range(len(candidate_app_elements)):
      input_ids_decoder.append(candidate_apps_tokenized["input_ids"][candidate_app_index])
      attention_mask_decoder.append(candidate_apps_tokenized["attention_mask"][candidate_app_index])
      input_ids_encoder.append(prompts_tokenized["input_ids"][index])
      attention_mask_encoder.append(prompts_tokenized["attention_mask"][index])
  
  input_ids_encoder_batches = list(chunk(input_ids_encoder, batch_size))
  attention_mask_encoder_batches = list(chunk(attention_mask_encoder, batch_size))
  input_ids_decoder_batches = list(chunk(input_ids_decoder, batch_size))
  attention_mask_decoder_batches = list(chunk(attention_mask_decoder, batch_size))
  

  scores = []
  for input_ids_encoder_batch, attention_mask_encoder_batch, input_ids_decoder_batch, attention_mask_decoder_batch in tqdm(zip(input_ids_encoder_batches, attention_mask_encoder_batches, input_ids_decoder_batches, attention_mask_decoder_batches), total = len(input_ids_encoder_batches)):
    decoder_input_ids = torch.stack(input_ids_decoder_batch).to("cuda")
    decoder_attention_mask = torch.stack(attention_mask_decoder_batch).to("cuda")
    input_ids = torch.stack(input_ids_encoder_batch).to("cuda")
    attention_mask = torch.stack(attention_mask_encoder_batch).to("cuda")
    with torch.no_grad():
      model_output = model(decoder_input_ids=decoder_input_ids, decoder_attention_mask=decoder_attention_mask, 
                           input_ids=input_ids, attention_mask=attention_mask)
    
    logprobs = F.log_softmax(model_output["logits"], dim=-1)[:, :-1, :] # remove the eos token
    output_tokens = decoder_input_ids[:, 1:] # remove the bos token
        
    tokens_logprobs = torch.gather(logprobs, 2, output_tokens[:, :, None]).squeeze(-1).to(torch.float32)
        
    mask = torch.ones(tokens_logprobs.shape, dtype=torch.bool, device="cuda")
    for i, _output in enumerate(output_tokens):
      for j, _token in enumerate(_output):
        if _token == tokenizer.pad_token_id:
          mask[i, j] = False
              
    score = (tokens_logprobs * mask).sum(-1) / mask.sum(-1)
    scores.extend(score.to('cpu').tolist())
    
  # batch_input_representations = torch.cat(batch_input_representations)
  
  scores = convert_to_sublists(scores, len(candidate_apps[0]))
  
  return scores

In [23]:
scores = recommender_rank(prompt_test, candidate_books, model, tokenizer, batch_size=8)

100%|██████████| 5497/5497 [20:40<00:00,  4.43it/s]


Sampled Candidates

In [24]:
# that is the one
[top_k_accuracy_score(true_candidate_indexes, scores, k=k) for k in range(1, 11)]

[np.float64(0.7009664582148948),
 np.float64(0.8032973280272883),
 np.float64(0.8573052870949404),
 np.float64(0.8908470722001137),
 np.float64(0.9198408186469585),
 np.float64(0.939169982944855),
 np.float64(0.9511085844229676),
 np.float64(0.9596361569073337),
 np.float64(0.9670267197271177),
 np.float64(0.9749857873791927)]

In [25]:
true_relevance = [[1 if item == index else 0 for item in range(len(candidate_books[0]))] for index in true_candidate_indexes]

In [26]:
# that is the one
[ndcg_score(true_relevance, scores, k=k) for k in range(1, 11)]

[np.float64(0.7009664582148948),
 np.float64(0.7655300486883811),
 np.float64(0.7925340282222072),
 np.float64(0.8069796887829405),
 np.float64(0.8181960009881489),
 np.float64(0.8250811882318516),
 np.float64(0.829060722057889),
 np.float64(0.831750871660951),
 np.float64(0.8339756527545449),
 np.float64(0.8362763392630443)]

Similar Candidates

In [ ]:
[top_k_accuracy_score(true_candidate_indexes, scores, k=k) for k in range(1, 11)]

In [44]:
true_relevance = [[1 if item == index else 0 for item in range(len(candidate_books[0]))] for index in true_candidate_indexes]

In [ ]:
[ndcg_score(true_relevance, scores, k=k) for k in range(1, 11)]